In [1]:
import numpy as np
import pytorch_lightning as pl

from src.models.high_level_model import HighLevelModel
from src.calibration.calibration import calibration
from src.data.multi_output_dataset import MultiOutputDataModule
from src.models.model_utils import convert_multitask_preds

In [2]:
MDC_COLOR = 12
MDC_TYPE = 11
MDC_TASK_NUM_CLASSES = [MDC_COLOR, MDC_TYPE]
root_dir = "data"

In [3]:
model = HighLevelModel.load_from_checkpoint(
    "models/mdc-high-level-model.ckpt",
    map_location="cpu",
    task_num_classes=MDC_TASK_NUM_CLASSES,
)
datamodule = MultiOutputDataModule(
    root_dir=root_dir,
    task_num_classes=MDC_TASK_NUM_CLASSES,
    batch_size=64,
    num_workers=8,
)
datamodule.setup()

In [4]:
model.eval()
trainer = pl.Trainer(accelerator="gpu")

calib_preds = trainer.predict(model, dataloaders=datamodule.calib_dataloader())
calib_preds = convert_multitask_preds(calib_preds)
true_labels = [labels for _, labels in datamodule.datasets["calib"]]

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/tmb2k01/masters-thesis/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off prec

Predicting DataLoader 0: 100%|██████████| 76/76 [00:11<00:00,  6.62it/s]


In [5]:
q_hats = calibration(
    calib_preds,
    true_labels,
    high_level=isinstance(model, HighLevelModel),
)

AttributeError: 'list' object has no attribute 'ndim'